# The Adding Problem

The "adding problem" was original proposed by Schmidhuber and colleagues as an example of a sequential task that LSTM's are particularly well suited for: http://people.idsia.ch/~juergen/nipslstm/node4.html

Here we will consider a slightly modified version where the sequence is of fixed length T and the sequence has two dimensions. The first dimension is a random number uniformly from 0 to 1 and the second dimension is always 0 except for two random positions and acts as a mask to add the numbers in the first dimension. 

>![The Adding Problem](https://minpy.readthedocs.io/en/latest/_images/adding_problem.png)

As another example, the following sequence of length 5
```
{(0.443, 0), 
 (0.112, 1), 
 (0.950, 0), 
 (0.839, 1), 
 (0.142, 0)} 
 ```

yields 0.112 + 0.839 = 0.951 as the answer since the 2nd and 4th elmemts are added. 

Here we will compare several different cells in PyTorch to see how well they solve the adding problem. The cells we consider are:

- RNN
- LSTM
- GRU

We will also consider a convolutional layer. Conv1D is not a recurrent layer, but has been shown to me useful for some sequential tasks. 

All methods will be compared using MSE on a held out test set. 

In [1]:
!pip install torch

    100% |████████████████████████████████| 519.5MB 30kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x597e0000 @  0x7fada90b61c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8


In [1]:
import numpy as np
import pandas as pd

import matplotlib.pylab as plt
import seaborn as sns;
%matplotlib inline

In [2]:
import sys
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [3]:
# use CUDA or not
use_cuda = True
if use_cuda and torch.cuda.is_available():
  print("using cuda!")
  device = torch.device("cuda")
else:
  print("using CPU!")

using CPU!


## Data loading functions

We will define some helper functions to generate our datasets. `generate_sequence` will genrate a single sequence whereas `get_set` returns multiple sequences (so a *dataset* of sequences).



In [4]:
def generate_sequence(seq_len = 10):
  ''' generate sequences
  
  Args:
  -----
  seq_len : int (default 10)
    The length of the sequence
  
  Returns:
  --------
  tuple of 3 numpy arrays x, z, y. x and z are 1D arrays and have same length
  y is a float that is the target we want to predict (addition of x masked by z)
  
  Example:
  --------
  
  >>> x_seq, z_seq, y_target = generate_sequence(seq_len = 100)
  
  '''
  x = np.random.rand(seq_len)
  z_p = np.arange(seq_len)
  np.random.shuffle(z_p)
  z = np.zeros(seq_len)
  z[z_p[0]] = 1
  z[z_p[1]] = 1
  y = x[z_p[0]] + x[z_p[1]]
  return x, z, y

def get_set(num_examples = 100, seq_len = 10):
  '''
  Get the data set used for training/testing networks.
  
  Args:
  -----
  num_examples : int (default 100)
    Number of sequences to generate
  
  seq_len : int (default 10)
    The length of the sequence
    
  Returns:
  --------
  typle of length 2 where the first tuple is a numpy array of shape 
  num_examples x seq_len x 2 and the second tuple is length num_examples
  
  Example:
  --------
  
  >>> X, y = get_set(num_examples=1000, seq_len = 50)
  
  '''
  X_set, Z_set, y_set = [], [], []

  for _ in range(num_examples):
    x_example, z_example, y_example = generate_sequence(seq_len)
    X_set.append(x_example)
    Z_set.append(z_example)
    y_set.append(y_example)
    
  X = np.zeros((num_examples,seq_len,2))
  X[:,:,0] = np.array(X_set)
  X[:,:,1] = np.array(Z_set)
  return X, np.array(y_set)  

Lets see `get_set` in action:

In [5]:
X_train, y_train = get_set(num_examples=100, seq_len = 10)
X_test, y_test = get_set(num_examples=100, seq_len = 10)
X_train.shape

(100, 10, 2)

So for the input we have a 3D array that has shape "num examples" x "sequence length" x "num features."


Note that the datasets that `get_set` returns are Numpy arrays, but PyTorch recquires PyTorch tensors. We could of course convert these Numpy arrays to PyTorch arrays, and then do some booking with indices to keep track of going through different batches when doing batch updates on the network.

But that is tedious and PyTorch offers the Dataset class that we can inherit from to keep all this bookkeeping for us. Below we define the `SequenceDataset` generator class that will be used for all our data handilng for PyTorch. 

In [6]:
class SequenceDataset(Dataset):
  
  def __init__(self, num_examples, seq_len):
    self.num_examples = num_examples
    self.seq_len = seq_len
    
    X, y = get_set(num_examples=self.num_examples, seq_len = self.seq_len)
    self.X = torch.from_numpy(X).float()
    self.y = torch.from_numpy(y).float()
    if use_cuda and torch.cuda.is_available():
      self.X = self.X.to(device)
      self.y = self.y.to(device)
    
    
    
  def __getitem__(self, index):
    return self.X[index], self.y[index]
  
  def __len__(self):
    return self.num_examples

  

Lets create a training and test set with 100 examples for each and sequence lengths of 10. 

In [7]:
train_set = SequenceDataset(num_examples=100, seq_len = 10)
test_set = SequenceDataset(num_examples=100, seq_len = 10)



We can use PyTorch's `DataLoader` to specify the the batches of data to load for training. Note that each of the 100 example sequences are independent, so we also shuffle the order of the different sequences. 


In [8]:
batch_size = 32

train_loader = DataLoader(dataset = train_set,
                          batch_size=batch_size,
                          shuffle = True)

test_loader = DataLoader(dataset = test_set,
                         batch_size=batch_size,
                         shuffle = True)

## RNN

We will start solving the Adding Problem with a simple RNN (the *Elman Network*). The network will update its internal hidden state for every element in the sequence until we reach the end. When we reach the end, we pass the final hidden state through a fully connected linear layer to predict the target. This type of architecture is sometimes called *many-to-one* since we are taking "many" elements (a sequence) to a single element (the target).

<center>
![Many to one](https://i.stack.imgur.com/QCnpU.jpg)
</center>

In [9]:
class RNNAdder(nn.Module):

    def __init__(self, hidden_size, input_size):    
        super(RNNAdder, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size 
        
        self.rnn = nn.RNN(input_size=self.input_size,
                          hidden_size=self.hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # Initialize hidden state. The shape of the tensor is
        # (num_layers * num_directions, batch, hidden_size)
        h_0 = Variable(torch.zeros(1, x.size(0), self.hidden_size))
        # since our input has the batch dimension in the first dim, 
        # we just use x.size(0)        
        if use_cuda and torch.cuda.is_available():
          h_0 = h_0.to(device)

        # Propagate input through RNN
        # Input: (batch, seq_len, embedding_size)
        _, h_f = self.rnn(x, h_0)
        # we only care about the final hidden state. The intermediate values 
        # of the hidden state are discarded. We pass the final hidden state
        # through the fully connected linear layer
        return self.fc(h_f).squeeze()


In [10]:
rnn_adder = RNNAdder(hidden_size = 12, input_size = 2)

if use_cuda and torch.cuda.is_available():
    rnn_adder = rnn_adder.cuda(device)

In [11]:
# Set loss and optimizer function
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(rnn_adder.parameters(), lr=0.01)

In [13]:
%%time
num_epochs = 100
for epoch in range(num_epochs):
  for i, (sequences, targets) in enumerate(train_loader):
#     if use_cuda and torch.cuda.is_available():
#       sequences = sequences.to(device)
#       targets = targets.to(device)

    
    # forward pass
    outputs = rnn_adder(sequences)
    loss = criterion(outputs, targets)
    
    # update weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
  if (epoch+1)%100 == 0:
    print("loss is", loss.item())

loss is 0.13951006531715393
CPU times: user 13.5 s, sys: 142 ms, total: 13.6 s
Wall time: 6.39 s


In [14]:
with torch.no_grad():
  outputs = rnn_adder(test_set.X)
  test_mse = torch.mean((outputs - test_set.y)**2)
print(test_mse.item())

0.16553369164466858


## LSTM

RNN's suffer from the vanishing gradient problem since creating the final hidden state is a result of updating the state through multiplications everytime a new element arrives in the sequence. LSTM's bypass this challenge by updating state additively. As a result, updaing gradients is much easier and longer memories can persist. Below is an `LSTMAdder` that is nearly identical to the `RNNAdder.`



In [15]:
class LSTMAdder(nn.Module):

    def __init__(self, hidden_size, input_size):    
        super(LSTMAdder, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size 
        self.lstm = nn.LSTM(input_size=self.input_size,
                          hidden_size=self.hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # Initialize hidden and cell states
        # (num_layers * num_directions, batch, hidden_size)
        h_0 = Variable(torch.zeros(1, x.size(0), self.hidden_size))
        c_0 = Variable(torch.zeros(1, x.size(0), self.hidden_size))
        if use_cuda and torch.cuda.is_available():
          h_0 = h_0.to(device)
          c_0 = c_0.to(device)

        # Propagate input through LSTM
        # Input: (batch, seq_len, embedding_size)
        # h_0: (num_layers * num_directions, batch, hidden_size)
        _, (h_f, c_f) = self.lstm(x, (h_0, c_0))
        return self.fc(h_f).squeeze()


In [16]:
lstm_adder = LSTMAdder(hidden_size = 12, input_size = 2)
if use_cuda and torch.cuda.is_available():
    lstm_adder = lstm_adder.cuda(device)

In [17]:
# Set loss and optimizer function
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(lstm_adder.parameters(), lr=0.01)

In [18]:
%%time
num_epochs = 1000
for epoch in range(num_epochs):
  for i, (sequences, targets) in enumerate(train_loader):
    # forward pass
    outputs = lstm_adder(sequences)
    loss = criterion(outputs, targets)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
  if (epoch+1)%100 == 0:
    print("loss is", loss.item())

loss is 0.2949182987213135
loss is 0.000536929932422936
loss is 4.7254226956283674e-05
loss is 0.00031535691232420504
loss is 0.00027109982329420745
loss is 0.0002998627896886319
loss is 4.688739136327058e-05
loss is 0.0002934955118689686
loss is 0.00018490737420506775
loss is 8.480088581563905e-06
CPU times: user 9min 26s, sys: 4.72 s, total: 9min 30s
Wall time: 4min 49s


In [ ]:
with torch.no_grad():
  outputs = lstm_adder(test_set.X)
  test_mse = torch.mean((outputs - test_set.y)**2)
print(test_mse.item())

## ReLU RNN

The idea of the ReLU RNN is to initialize the hidden state of the RNN with the identity matrix and the bias with 0 and use the ReLU activation function. Below we demonstrate how such an RNN can be implemented. The results are not as good as the LSTM but certainly better than the traditional Elman Network.

In [0]:
class ReLURNNAdder(nn.Module):

    def __init__(self, hidden_size, input_size):    
        super(ReLURNNAdder, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size 
        
        self.rnn = nn.RNN(input_size=self.input_size,
                          hidden_size=self.hidden_size, nonlinearity = "relu",
                          batch_first=True)
        
        torch.nn.init.zeros_(self.rnn.weight_ih_l0)
        torch.nn.init.eye_(self.rnn.weight_hh_l0)
        
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # Initialize hidden state. The shape of the tensor is
        # (num_layers * num_directions, batch, hidden_size)
        h_0 = Variable(torch.zeros(1, x.size(0), self.hidden_size))
        # since our input has the batch dimension in the first dim, 
        # we just use x.size(0)        
        if use_cuda and torch.cuda.is_available():
          h_0 = h_0.to(device)

        # Propagate input through RNN
        # Input: (batch, seq_len, embedding_size)
        _, h_f = self.rnn(x, h_0)
        # we only care about the final hidden state. The intermediate values 
        # of the hidden state are discarded. We pass the final hidden state
        # through the fully connected linear layer
        return self.fc(h_f).squeeze()


We could train this model as before but if we want to be fair in our comparisons,  we should train each adder for each epoch for each batch. This can help us control the differences in training procedures. 

In [0]:
relu_rnn_adder = ReLURNNAdder(hidden_size = 12, input_size = 2)
rnn_adder = RNNAdder(hidden_size = 12, input_size = 2)
lstm_adder = LSTMAdder(hidden_size = 12, input_size = 2)


if use_cuda and torch.cuda.is_available():
    relu_rnn_adder = relu_rnn_adder.cuda(device)
    rnn_adder = rnn_adder.cuda(device)
    lstm_adder = lstm_adder.cuda(device)

In [0]:
# Set loss and optimizer function
criterion = torch.nn.MSELoss()
relu_rnn_opt = torch.optim.Adam(relu_rnn_adder.parameters(), lr=0.01)
rnn_opt = torch.optim.Adam(rnn_adder.parameters(), lr=0.01)
lstm_opt = torch.optim.Adam(lstm_adder.parameters(), lr=0.01)


In [0]:
def update_model(model, optimizer, input_sequences, output_targets):
  preds = model(input_sequences)
  loss = criterion(preds, output_targets)
  
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  return loss.item()

In [22]:
%%time
lstm_losses = []
rnn_losses = []
relu_rnn_losses = []
for epoch in range(5000):
  for i, (sequences, targets) in enumerate(train_loader):
    
    loss = update_model(relu_rnn_adder, relu_rnn_opt, sequences, targets)
    relu_rnn_losses.append(loss)
    
    loss = update_model(rnn_adder, rnn_opt, sequences, targets)
    rnn_losses.append(loss)

    loss = update_model(lstm_adder, lstm_opt, sequences, targets)
    lstm_losses.append(loss)

    
  if (epoch+1)%100 == 0:
    print("LSTM loss:{:.2e}".format(lstm_losses[-1]) , 
          "RNN loss:{:.2e}".format(rnn_losses[-1]), 
          "ReLURNN loss:{:.2e}".format(relu_rnn_losses[-1]))

LSTM loss:7.84e-03 RNN loss:1.03e-01 ReLURNN loss:3.34e-01
LSTM loss:2.30e-04 RNN loss:5.75e-02 ReLURNN loss:4.10e-02
LSTM loss:1.67e-04 RNN loss:4.40e-03 ReLURNN loss:2.56e-02
LSTM loss:5.46e-04 RNN loss:3.14e-03 ReLURNN loss:1.14e-02
LSTM loss:4.40e-04 RNN loss:4.15e-04 ReLURNN loss:7.64e-03
LSTM loss:4.65e-04 RNN loss:2.17e-03 ReLURNN loss:1.93e-02
LSTM loss:8.30e-05 RNN loss:5.40e-03 ReLURNN loss:1.21e-03
LSTM loss:1.09e-04 RNN loss:5.69e-04 ReLURNN loss:5.51e-04
LSTM loss:3.47e-04 RNN loss:1.71e-04 ReLURNN loss:2.05e-03
LSTM loss:9.31e-05 RNN loss:1.15e-03 ReLURNN loss:3.66e-03
LSTM loss:1.52e-04 RNN loss:7.90e-04 ReLURNN loss:5.76e-03
LSTM loss:4.05e-04 RNN loss:1.50e-02 ReLURNN loss:6.46e-03
LSTM loss:3.09e-05 RNN loss:3.69e-04 ReLURNN loss:5.80e-04
LSTM loss:7.03e-06 RNN loss:5.70e-04 ReLURNN loss:1.01e-03
LSTM loss:6.47e-05 RNN loss:9.25e-04 ReLURNN loss:1.42e-03
LSTM loss:1.15e-04 RNN loss:7.27e-04 ReLURNN loss:1.14e-04
LSTM loss:2.26e-04 RNN loss:4.90e-04 ReLURNN loss:1.54e-

In [0]:
with torch.no_grad():
  outputs = relu_rnn_adder(test_set.X)
  relu_rnn_mse = torch.mean((outputs - test_set.y)**2)

  outputs = rnn_adder(test_set.X)
  rnn_mse = torch.mean((outputs - test_set.y)**2)

  outputs = lstm_adder(test_set.X)
  lstm_mse = torch.mean((outputs - test_set.y)**2)
  
  


In [24]:
lstm_mse.item()

0.0006150761619210243

In [25]:
rnn_mse.item()

0.05440850928425789

In [26]:
relu_rnn_mse.item()

0.0014745582593604922

While the LSTM is still the superior adder, the RNN initialized with the identity matrix and using the ReLU function is definitely better than the traidtional RNN.
